In [7]:
from selenium import webdriver
import time
import cv2
import numpy as np
from PIL import Image

In [8]:
#dict_portos = {'santos':'https://www.marinetraffic.com/en/ais/home/centerx:-46.314/centery:-23.964/zoom:13'}

In [9]:
dict_portos = {'santos':'https://www.marinetraffic.com/en/ais/home/centerx:-46.367/centery:-23.961/zoom:12'}

In [6]:
def clica_botao(driver, xpath):
    '''
    clica em um elemento da página e espera
    '''
    try:
        driver.find_element_by_xpath(xpath).click()
        time.sleep(3)
    except:
        pass
    
def move_offset(webdriver):
    '''
    move o mouse
    '''
    try:
        action = webdriver.ActionChains(driver)
        action.move_by_offset(10, 20)
        action.perform()
    except:
        pass
    
def fecha_add(driver):
    '''
    fecha propagandas do site
    '''
    try:
        # fecha add 1
        driver.find_element_by_xpath('//*[@id="leadinModal-405037"]/div[2]/button').click()
        time.sleep(3)

        # fecha add 2
        driver.find_element_by_xpath('//*[@id="div-gpt-ad-1539340679150-0"]').click()
        time.sleep(3)
    except:
        pass

# botao de accept
def clica_accept(driver):
    '''
    tenta clicar na mensagem de aceitar
    '''
    try:
        driver.find_element_by_xpath('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
        time.sleep(3)
    except:
        pass
    
# salva screenshot
def salva_screenshot(driver, imagem):
    '''
    salva screenshot da tela
    '''
    screenshot = driver.save_screenshot(imagem)
    img = cv2.imread(imagem)
    img = cv2.resize(img, (600, 500))
    cv2.imwrite(imagem, img)

    img = Image.open(imagem)
    crop_dim = (50, 50, 430, 410)
    cropped_img = img.crop(crop_dim)
    cropped_img.save(imagem)
    
# contagem de navios
def contagem_navios_img(img):
    '''
    faz a contagem de navios na imagem
    '''
    try:
        img = cv2.imread(img, 0)
        seed_pt = (25, 25)
        fill_color = 0
        mask = np.zeros_like(img)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        for th in range(60, 120):
            prev_mask = mask.copy()
            mask = cv2.threshold(img, th, 255, cv2.THRESH_BINARY)[1]
            mask = cv2.floodFill(mask, None, seed_pt, fill_color)[1]
            mask = cv2.bitwise_or(mask, prev_mask)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        qtd_navios = int(cv2.connectedComponents(mask)[0] - 1)
        return (qtd_navios)
    except:
        return 0

# processamento de imagem
def processamento_imagem(imagem_input, imagem_output):
    '''
    processamento da imagem
    '''
    max_amarelo = np.array([230, 230, 230])
    min_amarelo = np.array([0, 0, 0])

    img = cv2.imread(imagem_input)

    # convert BGR to HSV
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # create the Mask
    mask = cv2.inRange(imgHSV, min_amarelo, max_amarelo)

    # inverse mask
    mask = 255-mask
    img = cv2.bitwise_and(img, img, mask=mask)

    cv2.imwrite(imagem_output, img)

    # corte em branco
    # limites superiores e inferiores
    lower = np.array([230, 230, 230])
    upper = np.array([255, 255, 255])

    # máscara para selecionar apenas a cor preta
    thresh = cv2.inRange(img, lower, upper)

    # aplica metodologia
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # inverte máscara
    mask = 255 - morph

    # aplica máscara na imagem
    result = cv2.bitwise_and(img, img, mask=mask)

    # salva imagem
    cv2.imwrite(imagem_output, result)

    img = cv2.imread(imagem_output)
    img_bw = 255*(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) > 5).astype('uint8')
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    mask = cv2.morphologyEx(img_bw, cv2.MORPH_OPEN, se2)
    mask = np.dstack([mask, mask, mask]) / 255
    img = (img * mask)

    # salva imagem
    cv2.imwrite(imagem_output, img)


def extrai_imagem_site(driver):
    '''
    extrai a imagem do site
    '''
    # botao de accept
    clica_accept(driver)

    # abre layers
    clica_botao(driver, '//*[@id="svg_icon_layers"]')

    # tipo de mapa
    clica_botao(driver, '//*[@id="baseLayerSelect"]')

    # mapa tipo satelite
    clica_botao(driver, '//*[@id="baseLayerSelect"]/option[3]')

    # fecha layers
    clica_botao(driver, '//*[@id="map_area_outer"]/div[2]/div[1]/div/div[1]/div[9]/div[1]')

    # fecha add
    fecha_add(driver)

    # move offset
    move_offset(webdriver)

    # salva screenshot
    salva_screenshot(driver, f'imagens/img_satelite_{porto}.png')

    # abre layers
    clica_botao(driver, '//*[@id="svg_icon_layers"]')

    # tipo de mapa
    clica_botao(driver, '//*[@id="baseLayerSelect"]')

    # mapa tipo escuro
    clica_botao(driver, '//*[@id="baseLayerSelect"]/option[5]')

    # fecha layers
    clica_botao(driver, '//*[@id="map_area_outer"]/div[2]/div[1]/div/div[1]/div[9]/div[1]')

    # fecha add
    fecha_add(driver)

    # move offset
    move_offset(webdriver)

    # salva screenshot
    salva_screenshot(driver, f'imagens/img_dark_{porto}.png')

    # processamento da imagem
    processamento_imagem(f'imagens/img_dark_{porto}.png', f'imagens/img_contagem_navios_{porto}.png')

    # contagem de navios na imagem
    qtd_navios = contagem_navios_img(f'imagens/img_contagem_navios_{porto}.png')
    
    return qtd_navios



    
# main
chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument("--headless")

driver_path = 'chromedriver'
#driver = webdriver.Chrome(driver_path, options=chromeOptions)
driver = webdriver.Chrome(driver_path)

porto = 'santos'

url_porto = dict_portos[porto]

driver.get(url_porto)
time.sleep(5)

# extrai imagem do site
qtd_navios = extrai_imagem_site(driver)
print (qtd_navios)

<ipython-input-6-a5976969cacf>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
<ipython-input-6-a5976969cacf>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(xpath).click()
<ipython-input-6-a5976969cacf>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="leadinModal-405037"]/div[2]/button').click()
<ipython-input-6-a5976969cacf>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="div-gpt-ad-1539340679150-0"]').click()


WebDriverException: Message: chrome not reachable
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x01120C43+2493507]
	Ordinal0 [0x010BA4B1+2073777]
	Ordinal0 [0x00FC24D0+1058000]
	Ordinal0 [0x00FB7604+1013252]
	Ordinal0 [0x00FB7E06+1015302]
	Ordinal0 [0x00FB94D5+1021141]
	Ordinal0 [0x00FB35E3+996835]
	Ordinal0 [0x00FC3820+1062944]
	Ordinal0 [0x010147D2+1394642]
	Ordinal0 [0x0100580B+1333259]
	Ordinal0 [0x00FE2314+1188628]
	Ordinal0 [0x00FE316F+1192303]
	GetHandleVerifier [0x012A7BF6+1548950]
	GetHandleVerifier [0x0135461C+2256060]
	GetHandleVerifier [0x011AC13B+518107]
	GetHandleVerifier [0x011AB1E0+514176]
	Ordinal0 [0x010BF53D+2094397]
	Ordinal0 [0x010C3418+2110488]
	Ordinal0 [0x010C3552+2110802]
	Ordinal0 [0x010CCE81+2150017]
	BaseThreadInitThunk [0x75E6FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77BD7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77BD7A6E+238]


In [11]:
# main
chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument("--headless")

driver_path = 'chromedriver'
driver = webdriver.Chrome(driver_path, options=chromeOptions)
#driver = webdriver.Chrome(driver_path)

porto = 'santos'

url_porto = dict_portos[porto]

driver.get(url_porto)
time.sleep(10)

# extrai imagem do site
qtd_navios = extrai_imagem_site(driver)
print (qtd_navios)

<ipython-input-6-a5976969cacf>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
<ipython-input-6-a5976969cacf>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(xpath).click()
<ipython-input-6-a5976969cacf>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="leadinModal-405037"]/div[2]/button').click()
<ipython-input-6-a5976969cacf>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="div-gpt-ad-1539340679150-0"]').click()


51
